# Hybrid CF con Cosine e Pearson

In [784]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_absolute_error, mean_squared_error
from scipy import sparse
from scipy.sparse.linalg import svds
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from typing import Tuple, List
from mlxtend.frequent_patterns import apriori, association_rules

In [785]:
# Sezione 1: Caricamento dati (MovieLens 100k)
url = "../datasets/ml-100k/u.data"
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_csv(url, sep='\t', names=columns)
df.drop(columns='timestamp', inplace=True)

print(df.head())

   user_id  movie_id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1


In [786]:
# Sezione 2: Split train/test e train_matrix
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

In [787]:
# Sezione 3: Controlli i casi di cold start per user e movie
# Estrai gli insiemi di ID
train_users = set(train_data['user_id'].unique())
test_users  = set(test_data ['user_id'].unique())

train_movies = set(train_data['movie_id'].unique())
test_movies  = set(test_data ['movie_id'].unique())

# Trova quelli “cold” (presenti in test ma non in train)
cold_users  = test_users  - train_users
cold_movies = test_movies - train_movies

print(f"Utenti in test NON in train (cold users): {len(cold_users)}")  
print(cold_users)  # se vuoi la lista completa

print(f"Film in test NON in train (cold items): {len(cold_movies)}")  
print(cold_movies)

Utenti in test NON in train (cold users): 0
set()
Film in test NON in train (cold items): 51
{np.int64(1536), np.int64(1156), np.int64(1669), np.int64(1290), np.int64(907), np.int64(1546), np.int64(1674), np.int64(1675), np.int64(1551), np.int64(1677), np.int64(913), np.int64(1681), np.int64(1682), np.int64(1433), np.int64(1565), np.int64(1320), np.int64(1577), np.int64(1452), np.int64(1453), np.int64(814), np.int64(1583), np.int64(1201), np.int64(1587), np.int64(1460), np.int64(1596), np.int64(1599), np.int64(1600), np.int64(1601), np.int64(1603), np.int64(1476), np.int64(1349), np.int64(1352), np.int64(1611), np.int64(1614), np.int64(1619), np.int64(1364), np.int64(1493), np.int64(1494), np.int64(599), np.int64(1625), np.int64(1630), np.int64(1632), np.int64(1122), np.int64(1507), np.int64(1637), np.int64(1640), np.int64(1641), np.int64(1648), np.int64(1649), np.int64(1651), np.int64(1655)}


In [788]:
# Sezione 4: Rimuovo i casi di cold start e creo la situazione ideale
# 2) Filtra il test_data lasciando solo le righe “note” al train
test_data_clean = test_data[
    test_data['user_id'].isin(train_users) &
    test_data['movie_id'].isin(train_movies)
].copy()

# 3) Verifico che non ci siano più cold-start
cold_users  = set(test_data_clean['user_id'])  - train_users
cold_movies = set(test_data_clean['movie_id']) - train_movies
assert len(cold_users)  == 0, "Ci sono ancora utenti cold-start!"
assert len(cold_movies) == 0, "Ci sono ancora film cold-start!"


In [789]:
# Sezione 5: Matrice utente-item e sparsità
train_rating_matrix_raw = train_data.pivot_table(index='user_id', columns='movie_id', values='rating')
n_users, n_items = train_rating_matrix_raw.shape
nnz = train_rating_matrix_raw.count().sum()
sparsity = 1 - nnz / (n_users * n_items)
print(f"Numero utenti: {n_users}, Numero item: {n_items}")
print(f"Ratings non nulli: {nnz}")
print(f"Sparsità: {sparsity:.2%}")

# Trova film (colonne) che non sono mai stati votati nel train set
empty_movies = train_rating_matrix_raw.isna().all(axis=0)

# Conta quanti sono
print("Numero di film senza alcun voto nel train set:", empty_movies.sum())

# Trova user (righe) che non hanno mai votato nel train set
lazy_users = train_rating_matrix_raw.isna().all(axis=1)

# Conta quanti sono
print("Numero di utenti che non hanno votato nessun film:", lazy_users.sum())

Numero utenti: 943, Numero item: 1631
Ratings non nulli: 70000
Sparsità: 95.45%
Numero di film senza alcun voto nel train set: 0
Numero di utenti che non hanno votato nessun film: 0


In [790]:
# Sezione 6: Calcolo similarita per gli utenti con Pearson e per gli item con cosine

# Centra la matrice per utenti (riga)
R_centered = train_rating_matrix_raw.sub(train_rating_matrix_raw.mean(axis=1), axis=0)
R_filled_user = R_centered.fillna(0)

# Similarità utenti con Pearson 
sim_pearson_u = 1 - pairwise_distances(R_filled_user, metric='correlation')
user_similarity = pd.DataFrame(sim_pearson_u,
                               index=train_rating_matrix_raw.index,
                               columns=train_rating_matrix_raw.index)

# Transponi la matrice (items come righe) e riempi i NaN con 0
R_filled_item = train_rating_matrix_raw.T.fillna(0)

# Calcola la similarità cosine sugli item
sim_cosine_i = cosine_similarity(R_filled_item)

# Ricostruisci in DataFrame con gli stessi indici/colonne
item_similarity = pd.DataFrame(sim_cosine_i,
                               index=train_rating_matrix_raw.columns,
                               columns=train_rating_matrix_raw.columns)


In [791]:
print("Similarità utenti (Pearson):")
print(user_similarity)

print("\nSimilarità item (Cosine):")
print(item_similarity)


Similarità utenti (Pearson):
user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.025596 -0.011685  0.074057  0.051482  0.047608  0.106999   
2        0.025596  1.000000  0.008961  0.005184  0.018338  0.074488  0.043858   
3       -0.011685  0.008961  1.000000  0.051455  0.000000 -0.025470 -0.015642   
4        0.074057  0.005184  0.051455  1.000000 -0.004354 -0.047942 -0.002955   
5        0.051482  0.018338  0.000000 -0.004354  1.000000  0.013809  0.060387   
...           ...       ...       ...       ...       ...       ...       ...   
939      0.112058  0.029783  0.000000  0.000000  0.059318 -0.036390  0.006551   
940      0.024172  0.034334 -0.025716  0.053082 -0.006414 -0.006285 -0.035584   
941     -0.003368  0.002481  0.016064 -0.043519  0.043398  0.004211  0.022798   
942     -0.053440  0.006037  0.018054 -0.026840  0.031094  0.020078  0.073802   

In [792]:
# Numero di NaN nella matrice di similarità utenti (Pearson)
num_nan_user = user_similarity.isna().sum().sum()
print(f"Numero di NaN in user_similarity: {num_nan_user}")

# Numero di NaN nella matrice di similarità item (Cosine)
num_nan_item = item_similarity.isna().sum().sum()
print(f"Numero di NaN in item_similarity: {num_nan_item}")


Numero di NaN in user_similarity: 0
Numero di NaN in item_similarity: 0


In [793]:
# Sezione 4: Calcolo predizione per user

# 1. Media dei voti per ogni utente 
user_means = train_rating_matrix_raw.replace(0, np.nan).mean(axis=1)

# 2. Scarti rispetto alla media 
user_centered = train_rating_matrix_raw.sub(user_means, axis=0).fillna(0)

# 3. Numeratore: somma pesata degli scarti 
numerator_user = user_similarity.dot(user_centered) 

# 4. Denominatore: somma dei valori assoluti delle similarità 
denominator_user = np.abs(user_similarity).sum(axis=1) 

# 5. Divisione e aggiunta della media 
P_user = numerator_user.div(denominator_user, axis=0).add(user_means, axis=0)


In [794]:
# Sezione 5: Calcolo predizione per item

# 2. Numeratore: somma pesata degli scarti 
numerator_item = user_centered.dot(item_similarity) 

# 3. Denominatore: somma dei valori assoluti
denominator_item = np.abs(item_similarity).sum(axis=1)  

# 4. Divisione e aggiunta della media 
P_item = numerator_item.div(denominator_item, axis=1).add(user_means, axis=0)


In [795]:
# Sezione 6: matrice ibrida con tuning sui pesi

# Quanto pesare user-based vs item-based
alphas = np.arange(0, 1.1, 0.1)
hybrid_predictions = {}

for alpha in alphas:
    P_hybrid = alpha * P_user + (1 - alpha) * P_item
    hybrid_predictions[round(alpha, 2)] = P_hybrid
    print(f"Matrice ibrida creata per α = {round(alpha, 2)}")

Matrice ibrida creata per α = 0.0
Matrice ibrida creata per α = 0.1
Matrice ibrida creata per α = 0.2
Matrice ibrida creata per α = 0.3
Matrice ibrida creata per α = 0.4
Matrice ibrida creata per α = 0.5
Matrice ibrida creata per α = 0.6
Matrice ibrida creata per α = 0.7
Matrice ibrida creata per α = 0.8
Matrice ibrida creata per α = 0.9
Matrice ibrida creata per α = 1.0


In [796]:
P_hybrid = hybrid_predictions[0.5]

print("Valore minimo:", P_hybrid.min().min())
print("Valore massimo:", P_hybrid.max().max())

P_hybrid_rounded = P_hybrid.round().clip(1, 5)
P_hybrid_rounded.head()

Valore minimo: 1.4173942898008072
Valore massimo: 5.007945796814422


movie_id,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1670,1671,1672,1673,1676,1678,1679,1680
user_id,,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
4,4.0,5.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,5.0,...,5.0,5.0,5.0,5.0,4.0,4.0,4.0,5.0,5.0,5.0
5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [797]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, mean_absolute_error, mean_squared_error

# Serve per confrontare solo i rating effettivamente presenti nel test set
true_ratings = test_data_clean.copy()

results = []

for alpha, P_hybrid in hybrid_predictions.items():
    y_true = []
    y_pred = []

    for _, row in true_ratings.iterrows():
        user = row['user_id']
        item = row['movie_id']
        true_rating = row['rating']

        try:
            pred_rating = P_hybrid.loc[user, item]
        except KeyError:
            continue

        if np.isnan(pred_rating):
            continue

        y_true.append(true_rating)
        y_pred.append(pred_rating)

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    if len(y_true) == 0:
        continue

    for threshold in [2.0, 3.0, 4.0, 5.0]:
        y_true_bin = (y_true >= threshold).astype(int)
        y_pred_bin = (y_pred >= threshold).astype(int)

        results.append({
            'alpha': alpha,
            'threshold': threshold,
            'accuracy': accuracy_score(y_true_bin, y_pred_bin),
            'precision': precision_score(y_true_bin, y_pred_bin, zero_division=0),
            'recall': recall_score(y_true_bin, y_pred_bin, zero_division=0),
            'mae': mean_absolute_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred))
        })

# Crea DataFrame finale
results_df = pd.DataFrame(results)

# Visualizza
print("\n Risultati metriche per α e threshold ")
print(results_df.round(4))


--- Risultati metriche per α e threshold ---
    alpha  threshold  accuracy  precision  recall     mae    rmse
0     0.0        2.0    0.9417     0.9460  0.9947  0.8329  1.0402
1     0.0        3.0    0.8162     0.8560  0.9352  0.8329  1.0402
2     0.0        4.0    0.5245     0.7771  0.1954  0.8329  1.0402
3     0.0        5.0    0.7878     0.0000  0.0000  0.8329  1.0402
4     0.1        2.0    0.9416     0.9460  0.9945  0.8314  1.0385
5     0.1        3.0    0.8163     0.8567  0.9342  0.8314  1.0385
6     0.1        4.0    0.5256     0.7811  0.1964  0.8314  1.0385
7     0.1        5.0    0.7878     0.0000  0.0000  0.8314  1.0385
8     0.2        2.0    0.9416     0.9461  0.9945  0.8298  1.0369
9     0.2        3.0    0.8163     0.8570  0.9338  0.8298  1.0369
10    0.2        4.0    0.5267     0.7835  0.1981  0.8298  1.0369
11    0.2        5.0    0.7878     0.0000  0.0000  0.8298  1.0369
12    0.3        2.0    0.9417     0.9461  0.9945  0.8283  1.0352
13    0.3        3.0    0.8168

In [798]:
# 1. Combinazione che minimizza RMSE
best_rmse = results_df.loc[results_df['rmse'].idxmin()]

# 2. Combinazione che minimizza MAE
best_mae = results_df.loc[results_df['mae'].idxmin()]

# 3. Combinazione che massimizza Precision
best_precision = results_df.loc[results_df['precision'].idxmax()]

# 4. Combinazione che massimizza Recall
best_recall = results_df.loc[results_df['recall'].idxmax()]

# Stampa pulita
print("\n Combinazione con RMSE minimo:")
print(best_rmse[['alpha', 'threshold', 'rmse', 'mae', 'precision', 'recall']].round(4))

print("\n Combinazione con MAE minimo:")
print(best_mae[['alpha', 'threshold', 'rmse', 'mae', 'precision', 'recall']].round(4))

print("\n Combinazione con Precisione massima:")
print(best_precision[['alpha', 'threshold', 'rmse', 'mae', 'precision', 'recall']].round(4))

print("\n Combinazione con Recall massima:")
print(best_recall[['alpha', 'threshold', 'rmse', 'mae', 'precision', 'recall']].round(4))



 Combinazione con RMSE minimo:
alpha        1.0000
threshold    2.0000
rmse         1.0247
mae          0.8183
precision    0.9463
recall       0.9947
Name: 40, dtype: float64

 Combinazione con MAE minimo:
alpha        1.0000
threshold    2.0000
rmse         1.0247
mae          0.8183
precision    0.9463
recall       0.9947
Name: 40, dtype: float64

 Combinazione con Precisione massima:
alpha        0.8000
threshold    5.0000
rmse         1.0276
mae          0.8210
precision    1.0000
recall       0.0002
Name: 35, dtype: float64

 Combinazione con Recall massima:
alpha        0.9000
threshold    2.0000
rmse         1.0261
mae          0.8196
precision    0.9462
recall       0.9947
Name: 36, dtype: float64


In [799]:
from collections import defaultdict

# Crea un dizionario user -> set di film già visti nel train
train_seen = train_data.groupby('user_id')['movie_id'].apply(set).to_dict()

thresholds = [2.0, 3.0, 4.0, 5.0]

# Per salvare metriche
results = defaultdict(list)

for alpha, P_hybrid in hybrid_predictions.items():
    # Prepariamo liste per precision/recall calcolo
    precisions = {thr: [] for thr in thresholds}
    recalls = {thr: [] for thr in thresholds}

    for user in test_data_clean['user_id'].unique():
        # Film visti nel train per questo user
        seen_items = train_seen.get(user, set())

        # Tutti i film nel test per questo utente
        user_test = test_data_clean[test_data_clean['user_id'] == user]

        # Prendi predizioni per tutti gli item (rimuovi quelli già visti nel train)
        user_preds = P_hybrid.loc[user].drop(labels=seen_items, errors='ignore')

        # Ordina per rating predetto decrescente
        user_preds_sorted = user_preds.sort_values(ascending=False)

        # Prendi i top N, esempio N=10
        N = 10
        top_n_items = user_preds_sorted.head(N).index

        # Vero rating nel test set solo per top N
        relevant_items = user_test[user_test['movie_id'].isin(top_n_items)]

        for thr in thresholds:
            # Item rilevanti sopra threshold (nel test)
            relevant_positive = set(relevant_items[relevant_items['rating'] >= thr]['movie_id'])
            # Predetti sopra threshold
            pred_positive = set(top_n_items)

            # Precision: quante predizioni corrette su tot predette
            prec = len(relevant_positive.intersection(pred_positive)) / max(len(pred_positive), 1)

            # Recall: quante predizioni corrette su tot rilevanti nel test per user
            all_relevant = set(user_test[user_test['rating'] >= thr]['movie_id'])
            rec = len(relevant_positive.intersection(all_relevant)) / max(len(all_relevant), 1)

            precisions[thr].append(prec)
            recalls[thr].append(rec)

    # Media su tutti gli utenti
    for thr in thresholds:
        mean_prec = np.mean(precisions[thr])
        mean_rec = np.mean(recalls[thr])
        results['alpha'].append(alpha)
        results['threshold'].append(thr)
        results['precision'].append(mean_prec)
        results['recall'].append(mean_rec)

# Converto in DataFrame per visualizzazione
import pandas as pd
results_df = pd.DataFrame(results)

print(results_df)


    alpha  threshold  precision    recall
0     0.0        2.0   0.002757  0.000783
1     0.0        3.0   0.002227  0.000773
2     0.0        4.0   0.001591  0.000862
3     0.0        5.0   0.000424  0.000900
4     0.1        2.0   0.042206  0.025717
5     0.1        3.0   0.041463  0.028886
6     0.1        4.0   0.037646  0.039110
7     0.1        5.0   0.025027  0.056281
8     0.2        2.0   0.123118  0.059659
9     0.2        3.0   0.121421  0.066886
10    0.2        4.0   0.109544  0.088791
11    0.2        5.0   0.072428  0.133009
12    0.3        2.0   0.188865  0.082270
13    0.3        3.0   0.184411  0.090780
14    0.3        4.0   0.164793  0.116728
15    0.3        5.0   0.105302  0.177322
16    0.4        2.0   0.225239  0.092947
17    0.4        3.0   0.219830  0.102700
18    0.4        4.0   0.194698  0.132504
19    0.4        5.0   0.119618  0.197659
20    0.5        2.0   0.249099  0.098949
21    0.5        3.0   0.242948  0.109262
22    0.5        4.0   0.215164  0

In [800]:
# Sezione 12: Tuning peso ibridazione alpha (User vs Item)
# Proviamo diversi pesi alpha per P_hybrid = alpha * P_u + (1-alpha) * P_i
alphas = np.arange(0.0, 1.01, 0.1)
results = []
for alpha in alphas:
    # costruisci ibrido pesato
    P_h = alpha * P_u + (1 - alpha) * P_i
    # se DataFrame, converti in numpy array
    try:
        P_h_arr = P_h.to_numpy()
    except:
        P_h_arr = P_h
    # calcolo precision/recall@10
    precisions_a, recalls_a = [], []
    for u, actual in test_items.items():
        idx = user_to_idx.get(u)
        if idx is None:
            continue
        row = P_h_arr[idx]
        mask = np.ones_like(row, dtype=bool)
        seen_idx = [movie_to_idx[m] for m in train_items.get(u, set()) if m in movie_to_idx]
        mask[seen_idx] = False
        scores = row[mask]
        if scores.size < 10:
            continue
        topk = np.argpartition(-scores, 10)[:10]
        cands = np.arange(row.shape[0])[mask]
        top_items = set(cands[topk])
        hits = len(top_items & {movie_to_idx[m] for m in actual if m in movie_to_idx})
        precisions_a.append(hits / 10)
        recalls_a.append(hits / len(actual))
    if precisions_a:
        results.append((alpha, np.mean(precisions_a), np.mean(recalls_a)))
    else:
        results.append((alpha, None, None))
# Visualizza risultati
cv_df = pd.DataFrame(results, columns=['alpha', 'precision', 'recall'])
print(cv_df)

    alpha  precision    recall
0     0.0   0.058333  0.030831
1     0.1   0.058333  0.030831
2     0.2   0.058333  0.030831
3     0.3   0.058333  0.030831
4     0.4   0.058333  0.030831
5     0.5   0.058333  0.030831
6     0.6   0.058333  0.030831
7     0.7   0.058333  0.030831
8     0.8   0.058333  0.030831
9     0.9   0.058333  0.030831
10    1.0   0.058333  0.030831


In [801]:
# Trova combinazione che massimizza la precisione Top-N=10
best_precision_row = results_df.loc[results_df['precision'].idxmax()]
best_recall_row    = results_df.loc[results_df['recall'].idxmax()]

print(" Miglior Precision Top-N:")
print(f"Alpha: {best_precision_row['alpha']}, Threshold: {best_precision_row['threshold']}, Precision: {best_precision_row['precision']:.4f}, Recall: {best_precision_row['recall']:.4f}")

print("\n Miglior Recall Top-N:")
print(f"Alpha: {best_recall_row['alpha']}, Threshold: {best_recall_row['threshold']}, Recall: {best_recall_row['recall']:.4f}, Precision: {best_recall_row['precision']:.4f}")

 Miglior Precision Top-N:
Alpha: 0.9, Threshold: 2.0, Precision: 0.2630, Recall: 0.1025

 Miglior Recall Top-N:
Alpha: 0.8, Threshold: 5.0, Recall: 0.2104, Precision: 0.1336
